
resources: https://metis1.atlassian.net/l/cp/cKpbcXek

In [17]:
import requests

# VictoriaMetrics URL for importing data (replace with your actual VictoriaMetrics URL)
victoria_metrics_url = "http://localhost:8428/api/v1/import/prometheus"

# Sample metric data in the same format as the curl command
metrics_data = 'cpu_utilization{host="pg1"} 66.1'

try:
    # Send a POST request to insert metrics into VictoriaMetrics
    response = requests.post(victoria_metrics_url, data=metrics_data)
    
    # Check the response status code
    if response.status_code == 204:
        print("Metric inserted successfully.")
    else:
        print(f"Failed to insert metric. Status code: {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"An error occurred: {str(e)}")


Metric inserted successfully.


The CURL works. They Python should too. 

In [10]:
%%bash
curl -d 'cpu_utilization{host="pg1"} 66.6' -X POST 'http://localhost:8428/api/v1/import/prometheus'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    32    0     0  100    32      0   2662 --:--:-- --:--:-- --:--:--  4000


In [27]:
%%bash
curl -G 'http://localhost:8428/api/v1/export' -d 'match={__name__=~"cpu_utilization"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


100  1617  100  1617    0     0  33587      0 --:--:-- --:--:-- --:--:-- 36750


{"metric":{"__name__":"cpu_utilization","host":"pg4"},"values":[46.48,80.63,78.69,44.1,65.5,17.2],"timestamps":[1695234188306,1695234295250,1695234300387,1695234305561,1695234310744,1695234315923]}
{"metric":{"__name__":"cpu_utilization","host":"pg5"},"values":[93.1,27.45,67.33,20.05,68.17,70.72],"timestamps":[1695234188323,1695234295266,1695234300410,1695234305592,1695234310762,1695234315956]}
{"metric":{"__name__":"cpu_utilization","host":"pg2"},"values":[72.3,72.3,72.3,82.6,80.6,74.72,68.13,55.24,16.01,1.36],"timestamps":[1695226304525,1695226370810,1695226483447,1695234088873,1695234188269,1695234295219,1695234300340,1695234305490,1695234310709,1695234315866]}
{"metric":{"__name__":"cpu_utilization","host":"pg3"},"values":[36.13,16.56,80.39,33.67,37.45,37.89],"timestamps":[1695234188283,1695234295234,1695234300366,1695234305525,1695234310729,1695234315895]}
{"metric":{"__name__":"cpu_utilization","host":"pg1"},"values":[88.5,84.4,77.5,77.5,77.5,77.5,84.4,84.4,66.6,66.2,66.1,66.1,66

Trying to insert many rows. 
That works. Checked with the command ```curl -G 'http://localhost:8428/api/v1/export' -d 'match={__name__=~"cpu_utilization"}'```



Insert data every 5 seconds. 5 metrics for 6 PG servers (hosts)

In [26]:
import requests
import time
import random

# VictoriaMetrics URL for importing data (replace with your actual VictoriaMetrics URL)
victoria_metrics_url = "http://localhost:8428/api/v1/import/prometheus"

# Define metric names and labels
metric_names = ['cpu_utilization', 'free_memory', 'io_read', 'io_write', 'db_load']
hosts = ['pg1', 'pg2', 'pg3', 'pg4', 'pg5', 'pg6']  # Add four more hosts

try:
    for _ in range(5):  # Insert data 5 times
        start_time = time.time()  # Record the start time
    
        for host in hosts:
            for metric_name in metric_names:
                # Generate a random value between 0 and 100 for the metric
                metric_value = random.uniform(0, 100)
                
                # Create the metric string with random value and host label
                metric = f'{metric_name}{{host="{host}"}} {metric_value:.2f}'
                
                # Send a POST request to insert the metric into VictoriaMetrics
                response = requests.post(victoria_metrics_url, data=metric)

                # Check the response status code
                if response.status_code == 204:
                    print(f"Metric inserted successfully: {metric}")
                else:
                    print(f"Failed to insert metric: {metric}. Status code: {response.status_code}")
                    print(response.text)
        
        end_time = time.time()  # Record the end time
        
        # Calculate and print the time difference
        time_difference = end_time - start_time
        print(f"Time taken to send data: {time_difference:.2f} seconds")

        # Wait for 5 seconds before the next iteration
        time.sleep(5)
except Exception as e:
    print(f"An error occurred: {str(e)}")


Metric inserted successfully: cpu_utilization{host="pg1"} 40.42
Metric inserted successfully: free_memory{host="pg1"} 67.07
Metric inserted successfully: io_read{host="pg1"} 92.11
Metric inserted successfully: io_write{host="pg1"} 20.76
Metric inserted successfully: db_load{host="pg1"} 90.14
Metric inserted successfully: cpu_utilization{host="pg2"} 74.72
Metric inserted successfully: free_memory{host="pg2"} 14.22
Metric inserted successfully: io_read{host="pg2"} 25.15
Metric inserted successfully: io_write{host="pg2"} 53.93
Metric inserted successfully: db_load{host="pg2"} 47.96
Metric inserted successfully: cpu_utilization{host="pg3"} 16.56
Metric inserted successfully: free_memory{host="pg3"} 83.81
Metric inserted successfully: io_read{host="pg3"} 0.69
Metric inserted successfully: io_write{host="pg3"} 65.71
Metric inserted successfully: db_load{host="pg3"} 62.54
Metric inserted successfully: cpu_utilization{host="pg4"} 80.63
Metric inserted successfully: free_memory{host="pg4"} 1.57

Insert many rows to test Counter metric

In [33]:
import requests
import datetime

# VictoriaMetrics URL for querying data (replace with your actual VictoriaMetrics URL)
victoria_metrics_url = "http://localhost:8428/api/v1/query"

# Define the query to retrieve "rows_read" metric values for the last 24 hours
query = 'rows_read{host="pg1",db="db1"}'

# Calculate the start time as 24 hours ago from the current time
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(hours=24)

# Convert the start and end times to Unix timestamps (in milliseconds)
start_timestamp = int(start_time.timestamp()) * 1000
end_timestamp = int(end_time.timestamp()) * 1000

# Define the time range in the query
query += f'[{start_timestamp}:{end_timestamp}]'

try:
    # Send a GET request to query the metric from VictoriaMetrics
    response = requests.get(victoria_metrics_url, params={'query': query})

    # Print the full response content
    print(response.text)
    
    # Check the response status code
    if response.status_code != 200:
        print(f"Failed to query metric. Status code: {response.status_code}")
except Exception as e:
    print(f"An error occurred: {str(e)}")


{"status":"success","data":{"resultType":"matrix","result":[]},"stats":{"seriesFetched": "0"}}


Insert 5 data points for the metric rows_read. 
Notice!! We still need to learn how to send the original time, captured when fetching the dataset. 

In [ ]:

%%bash 
curl -d 'rows_read{host="pg1",db="db1"} 10000' -X POST 'http://localhost:8428/api/v1/import/prometheus'
sleep 5  # Wait for 5 seconds before the next run
curl -d 'rows_read{host="pg1",db="db1"} 12000' -X POST 'http://localhost:8428/api/v1/import/prometheus' 
sleep 5  
 curl -d 'rows_read{host="pg1",db="db1"} 13000' -X POST 'http://localhost:8428/api/v1/import/prometheus' 
sleep 5  
curl -d 'rows_read{host="pg1",db="db1"} 13000' -X POST 'http://localhost:8428/api/v1/import/prometheus'  
sleep 5
curl -d 'rows_read{host="pg1",db="db1"} 13000' -X POST 'http://localhost:8428/api/v1/import/prometheus' 
sleep 5
curl -d 'rows_read{host="pg1",db="db1"} 17000' -X POST 'http://localhost:8428/api/v1/import/prometheus'  

Query the data using the function ```increase```. You might need to change the time

In [34]:
%%bash
curl "http://localhost:8428/api/v1/query_range?query=increase(rows_read)&step=5s&start=2023-09-20T09:59:00.000Z&end=2023-09-21T10:17:00.000Z"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   256  100   256    0     0   7599      0 --:--:-- --:--:-- --:--:--  8258


{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"db":"db1","host":"pg1"},"values":[[1695236150,"10000"],[1695236155,"2000"],[1695236160,"1000"],[1695236165,"0"],[1695236170,"0"],[1695236175,"4000"]]}]},"stats":{"seriesFetched": "0"}}

In [36]:
import requests
import pandas as pd
from datetime import datetime

# VictoriaMetrics URL for querying data (replace with your actual VictoriaMetrics URL)
victoria_metrics_url = "http://localhost:8428/api/v1/query_range"

# Define the query parameters
query = 'increase(rows_read)'
step = '5s'
start_time = '2023-09-20T09:59:00.000Z'
end_time = '2023-09-21T10:17:00.000Z'

# Define query parameters as a dictionary
params = {
    'query': query,
    'step': step,
    'start': start_time,
    'end': end_time
}

try:
    # Send a GET request to query the metric data from VictoriaMetrics
    response = requests.get(victoria_metrics_url, params=params)

    # Check the response status code
    if response.status_code == 200:
        data = response.json()
        if 'data' in data and 'result' in data['data']:
            results = data['data']['result']
            if len(results) > 0:
                # Create a list to store data points
                data_points = []

                # Iterate through the results and convert timestamps to readable format
                for result in results:
                    values = result['values']
                    for value in values:
                        timestamp = datetime.utcfromtimestamp(value[0]).strftime('%Y-%m-%d %H:%M:%S')
                        data_points.append([timestamp, value[1]])

                # Create a DataFrame
                df = pd.DataFrame(data_points, columns=['Timestamp', 'Value'])

                # Print the DataFrame
                print(df)
            else:
                print("No data found for the query.")
        else:
            print("No data found for the query.")
    else:
        print(f"Failed to query metric. Status code: {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"An error occurred: {str(e)}")


             Timestamp  Value
0  2023-09-20 18:55:50  10000
1  2023-09-20 18:55:55   2000
2  2023-09-20 18:56:00   1000
3  2023-09-20 18:56:05      0
4  2023-09-20 18:56:10      0
5  2023-09-20 18:56:15   4000
